In [31]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
import re
import pandas as pd
import numpy as np
from glob import glob, iglob
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

Python 3.8.10 (default, May 26 2023, 14:05:08) 
[GCC 9.4.0] on linux


In [32]:
import utils
import gpt

LIBRARY = 'gs'
CATEGORY = 'jppy'
DATASET_FOLDER = f'./dataset/{LIBRARY}/{CATEGORY}/'
OUTPUT_FOLDER = './output/'

## meta & embeddings

In [33]:
df_meta = pd.read_parquet(OUTPUT_FOLDER + f'meta-{LIBRARY}-{CATEGORY}.parquet')
print(df_meta.shape)

(73, 9)


In [34]:
df_embedding = pd.read_parquet(OUTPUT_FOLDER + f'embedding-{LIBRARY}-{CATEGORY}.parquet')
print(df_embedding.shape)

(73, 1536)


In [35]:
df_meta.loc['42eeeb9eecacafa41adcb9e381fd99f664c0cde147972d16ac4d835215fbad15'].content

"# Acquisition of Qualification\nPurpose\nThis section explains the activities that you must carry out when an employee acquires Health Insurance and Employeefs\nPension status (for example; hiring).\nProcess Flow\n1. Register the employee ' s Social Insurance status in Social Insurance Basic Data Infotype (0140) . And in case of new hire,\nthe i nitial Standard Compensation in Social Insurance Premium Data Infotype (0141) must be maintained too.\n2. Output Acquisition of Qualification Form (printed form) or create Acquisition of Qualification Form Data File for date\nmedia submittion or electronic filing via the Internet.\nSee also:\nRegistrating in Infotype when Acquired\nAcquisition of Qualification Form (RPLDQOJ1)\nAcquisition Form Paper (RPLDQOJ2) – 2018 Onward\nAcquisition of Qualification Form Data File for SI Oﬃce (RPLSHOJ0)\nAcquisition of Qualification Form Data File for HI Union (RPLHUOJ0)\nAcquisition of Qualification Form Data File for Pens. Fund (RPLPFOJ0)\nAcquisition Fo

In [30]:
# file_path = df_meta.loc['42eeeb9eecacafa41adcb9e381fd99f664c0cde147972d16ac4d835215fbad15'].file_path
# content = df_meta.loc['42eeeb9eecacafa41adcb9e381fd99f664c0cde147972d16ac4d835215fbad15'].content
# print(file_path)
# print(utils.generate_checksum(file_path + content))

./dataset/gs/JPPY/jppyp1.md
7918a37865bcf619456ec368fe915becf10ed409fcfd5ef74d395088f1b171ce


## search embedding

In [5]:
cases = [
    "What is The Acquisition of Qualification Form"
]

In [6]:
embeddings = []
embedder = gpt.Embedding()
for item in cases:
    embeddings.append(embedder.run(item).result_)

In [7]:
np.array(embeddings).shape

(1, 1536)

In [8]:
print(embeddings)

[[-0.0069440980441868305, 0.002998989075422287, 0.009821292012929916, -0.04719303548336029, -0.04394516348838806, 0.036150261759757996, -0.017665617167949677, -0.009828352369368076, -0.019967371597886086, 0.00040576374158263206, 0.02606772817671299, -0.010117837227880955, 0.012920894660055637, 0.016437072306871414, -0.015872225165367126, -0.006421613972634077, 0.034455716609954834, -0.03578311204910278, -0.026293667033314705, -0.023088155314326286, -0.020857006311416626, 0.01213010773062706, -0.00921408087015152, -0.014460104517638683, 0.006820537615567446, -0.0037703593261539936, 0.005087160971015692, -0.029965177178382874, -0.009976625442504883, 0.014092953875660896, -0.0010387904476374388, 0.00010370253585278988, -0.0029725118074566126, -0.012490198016166687, -0.022735126316547394, -0.02207142859697342, -0.002695383271202445, -0.016366466879844666, 0.0333542637526989, -0.008500959724187851, 0.030699480324983597, 0.016069920733571053, -0.022819852456450462, 0.00879750493913889, 0.011

## similiarity by knn

In [9]:
req_index = 0
target = embeddings[req_index]
print(cases[req_index])

What is The Acquisition of Qualification Form


In [10]:
search_results = utils.semantic_search(target, df_embedding.values)

In [11]:
df_embedding.iloc[0].name

'60051d22c568c2d7d7d5feea79999fb63b612621530e557263ed15d01a014559'

## prompt engineering

In [12]:
prompts = []
for idx, score in search_results:
    meta_idx = df_embedding.iloc[int(idx)].name
    print(idx, meta_idx, score)
    
    plain_text = df_meta.loc[meta_idx].content
    print(plain_text)
    print('-'*100)

    prompts.append(plain_text)

45 42eeeb9eecacafa41adcb9e381fd99f664c0cde147972d16ac4d835215fbad15 0.8456761044501486
# Acquisition of Qualification
Purpose
This section explains the activities that you must carry out when an employee acquires Health Insurance and Employeefs
Pension status (for example; hiring).
Process Flow
1. Register the employee ' s Social Insurance status in Social Insurance Basic Data Infotype (0140) . And in case of new hire,
the i nitial Standard Compensation in Social Insurance Premium Data Infotype (0141) must be maintained too.
2. Output Acquisition of Qualification Form (printed form) or create Acquisition of Qualification Form Data File for date
media submittion or electronic filing via the Internet.
See also:
Registrating in Infotype when Acquired
Acquisition of Qualification Form (RPLDQOJ1)
Acquisition Form Paper (RPLDQOJ2) – 2018 Onward
Acquisition of Qualification Form Data File for SI Oﬃce (RPLSHOJ0)
Acquisition of Qualification Form Data File for HI Union (RPLHUOJ0)
Acquisition of

In [13]:
prompt_content = f"\n\n{'-'*10}\n\n".join(prompts)

## chat completion

In [14]:
print(prompt_content)

# Acquisition of Qualification
Purpose
This section explains the activities that you must carry out when an employee acquires Health Insurance and Employeefs
Pension status (for example; hiring).
Process Flow
1. Register the employee ' s Social Insurance status in Social Insurance Basic Data Infotype (0140) . And in case of new hire,
the i nitial Standard Compensation in Social Insurance Premium Data Infotype (0141) must be maintained too.
2. Output Acquisition of Qualification Form (printed form) or create Acquisition of Qualification Form Data File for date
media submittion or electronic filing via the Internet.
See also:
Registrating in Infotype when Acquired
Acquisition of Qualification Form (RPLDQOJ1)
Acquisition Form Paper (RPLDQOJ2) – 2018 Onward
Acquisition of Qualification Form Data File for SI Oﬃce (RPLSHOJ0)
Acquisition of Qualification Form Data File for HI Union (RPLHUOJ0)
Acquisition of Qualification Form Data File for Pens. Fund (RPLPFOJ0)
Acquisition Form Data File (RPL

In [15]:
from gpt import ChatGPT

In [16]:
content = f'''
```
{prompt_content}
```
Question: {cases[req_index]}
'''

In [ ]:
data = {
    "system": '''
    Welcome to our Knowledge Base Expert Chatbot, Codengine! 🤖 
    I am here to assist you with questions and answers based on the given information. 
    Please provide the relevant details enclosed within triple backticks (```) 
    and then ask your question by starting with "Question: ". I'll do my best to provide you with an accurate and helpful response within the scope of the provided data. 
    Have to answer the question in Japanese!!''',
    "messages": [{
        "user": "user",
        "text": content
    }]
}

In [18]:
response = ChatGPT(temperature=0).chat_completion(data)

In [19]:
response.choices[0].message.content

'質問：「The Acquisition of Qualification Form（資格取得申請書）」とは何ですか？'